In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# CREDITS FOR DATA CLEANING AND TEST SET PREPARATION GO TO DENIS LARIONOV's Kernel. I have simply modified them for the translated dataset.

#### This Kernel has produced a score of 1.14 with DecisionTreeRegressor,max_depth=11, no pruning.

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import pickle
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
import time
from xgboost import XGBRegressor
import string
from itertools import product
import gc

In [ ]:
test = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")

In [ ]:
import pandas as pd
item_categories = pd.read_csv("../input/future-sales-translated/item_categories-translated.csv")
items = pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
sales_train = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
sample_submission = pd.read_csv("../input/competitive-data-science-predict-future-sales/sample_submission.csv")
shops = pd.read_csv("../input/future-sales-translated/shops-translated.csv")
test = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")

In [ ]:
##Cleaning object data into dat_time and parsing
sales_train['date']=pd.to_datetime(sales_train['date'],format='%d.%m.%Y')

sales_train=sales_train.assign(day=sales_train['date'].dt.day,
                  month=sales_train['date'].dt.month,
                  year=sales_train['date'].dt.year)


In [ ]:
plt.figure(figsize=(10,5))
plt.xlim(-100,3000)
sns.boxplot(sales_train['item_cnt_day'])

plt.figure(figsize=(10,5))
plt.xlim(-100,45000)
sns.boxplot(sales_train['item_price'])

In [ ]:
sales_train=sales_train[sales_train['item_price']<40000]
sales_train=sales_train[sales_train['item_cnt_day']<1000]

In [ ]:
## one negative value, so we will drop it
print(sales_train['item_price'].sort_values())
sales_train=sales_train[(sales_train['item_price']<0)==False]

#### Shop name formatting

In [ ]:
for i in range(0,shops.shape[0]):
    temp=[x for x in shops['shop_name_translated'][i] if x not in string.punctuation]
    shops['shop_name_translated'][i]=''.join(temp)

shops['shop_city']=shops['shop_name_translated'].map(lambda x: str.split(x)[0])
shops['shop_type']=shops['shop_name_translated'].map(lambda x: str.split(x)[1] if len(str.split(x))>1 else str.split(x)[0])

shops['shop_city']=shops['shop_city'].apply(lambda x: x.strip())
shops['shop_type']=shops['shop_type'].apply(lambda x: x.strip())

shops['shop_city']=shops['shop_city'].apply(lambda x: x.lower())
shops['shop_type']=shops['shop_type'].apply(lambda x: x.lower())

Healing the data

In [ ]:
shops.loc[shops['shop_type'].isin(['tc','тц','shopping','tk']), 'shop_type']='tc'
shops.loc[shops['shop_type'].isin(['trc','trk']), 'shop_type']='trc'
shops.loc[shops['shop_type'].isin(['novgorod']), 'shop_type']='tc'  ## shopping which finally becomes TC
shops.loc[shops['shop_type'].isin(['posad']), 'shop_type']='tc'

label encoding the data

In [ ]:
my_encoder=LabelEncoder()
shops['shop_city']=my_encoder.fit_transform(shops['shop_city'])
shops['shop_type']=my_encoder.fit_transform(shops['shop_type'])

shops=shops[['shop_id','shop_city','shop_type']]

#### Item categories

In [ ]:
for i in range(0,item_categories.shape[0]):
    temp=[c for c in  item_categories['item_category_name_translated'][i] if c not in string.punctuation]
    item_categories['item_category_name_translated'][i]=''.join(temp)

cleaning

In [ ]:
item_categories['cat1']=item_categories['item_category_name_translated'].map(lambda x: str.split(x)[0])
item_categories['cat2']=item_categories['item_category_name_translated'].map(lambda x:  str.split(x)[1] if len(str.split(x))>1 
                                                                            else str.split(x)[0])

item_categories['cat1']=item_categories['cat1'].map(lambda x: x.lower())
item_categories['cat2']=item_categories['cat2'].map(lambda x:  x.lower())

encoding

In [ ]:
my_encoder=LabelEncoder()
item_categories['cat1']=my_encoder.fit_transform(item_categories['cat1'])
item_categories['cat2']=my_encoder.fit_transform(item_categories['cat2'])

item_categories=item_categories[['item_category_id','cat1','cat2']]

#### Items formatting

In [ ]:
items.drop('item_name',axis=1,inplace=True)

#### Homogenizing train and test data

we can see that the test set is a cross product of all the shops and items sold in the particular month.
We shall now do this to entire train data and merge them together

In [ ]:
sales_train['revenue']=sales_train['item_price']*sales_train['item_cnt_day']

In [ ]:
sales_train["max_item_price"]=sales_train.groupby("item_id")["item_price"].transform('max')

In [ ]:
## not really a mean, all fields have only one count. Done only to groupby for item_ids
max_price=sales_train.groupby("item_id")['max_item_price'].mean()

In [ ]:
print(test.item_id.nunique())
print(test.shop_id.nunique())
print(test.shape)

In [ ]:
group=sales_train.query("date_block_num<34").groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day':['sum']})
group.columns=['item_cnt_month']
group.reset_index(inplace=True)
group['item_cnt_month']=(group['item_cnt_month'].clip(0,20).astype(np.float16))

In [ ]:
matrix=[]

for i in range(0,34):
    shops_month=sales_train.loc[sales_train['date_block_num']==i,'shop_id'].unique()
    items_month=sales_train.loc[sales_train['date_block_num']==i,'item_id'].unique()
    matrix.append(np.array(list(product([i],shops_month,items_month))))
    
matrix=np.vstack(matrix)
matrix=pd.DataFrame(matrix,columns=['date_block_num','shop_id','item_id'])
test.drop('ID',axis=1,inplace=True)
test['date_block_num']=34
test=test[['date_block_num','shop_id','item_id']]
matrix=pd.concat([matrix,test],axis=0)

In [ ]:
matrix=pd.merge(matrix,group,how='left',on=['date_block_num','shop_id','item_id'])
matrix=pd.merge(matrix,items,how='left',on=['item_id'])
matrix=pd.merge(matrix,shops,how='left',on=['shop_id'])
matrix=pd.merge(matrix,item_categories,how='left',on=['item_category_id'])

In [ ]:
matrix["item_cnt_month"].fillna(0,inplace=True)

In [ ]:
a=matrix.groupby("date_block_num")["item_cnt_month"].mean()*matrix.groupby("date_block_num")["item_cnt_month"].count()
plt.plot(a.index,a.values)
plt.xlabel("date_block_num")
plt.ylabel("item_cnt_month")

In [ ]:
a=matrix.groupby(["date_block_num","shop_id"])["item_cnt_month"].sum().reset_index().query("shop_id<15")
sns.lineplot(x="date_block_num",y="item_cnt_month",data=a,hue=a.shop_id)
plt.xlabel("date_block_num")
plt.ylabel("item_cnt_month")

In [ ]:
criteria="shop_id>=15 and shop_id<30"
a=matrix.groupby(["date_block_num","shop_id"])["item_cnt_month"].sum().reset_index().query(criteria)
sns.lineplot(x="date_block_num",y="item_cnt_month",data=a,hue=a.shop_id)
plt.xlabel("date_block_num")
plt.ylabel("item_cnt_month")

In [ ]:
criteria="shop_id>=30 and shop_id<45"
a=matrix.groupby(["date_block_num","shop_id"])["item_cnt_month"].sum().reset_index().query(criteria)
sns.lineplot(x="date_block_num",y="item_cnt_month",data=a,hue=a.shop_id)
plt.xlabel("date_block_num")
plt.ylabel("item_cnt_month")

In [ ]:
criteria="shop_id>=45 and shop_id<60"
a=matrix.groupby(["date_block_num","shop_id"])["item_cnt_month"].sum().reset_index().query(criteria)
sns.lineplot(x="date_block_num",y="item_cnt_month",data=a,hue=a.shop_id)
plt.xlabel("date_block_num")
plt.ylabel("item_cnt_month")

In [ ]:
a=matrix.groupby(["date_block_num","cat1"])["item_cnt_month"].sum().reset_index()
sns.lineplot(x="date_block_num",y="item_cnt_month",data=a,hue=a.cat1)
plt.xlabel("date_block_num")
plt.ylabel("item_cnt_month")
plt.title("cat1")

In [ ]:
a=matrix.groupby(["date_block_num","cat2"])["item_cnt_month"].sum().reset_index()
sns.lineplot(x="date_block_num",y="item_cnt_month",data=a,hue=a.cat2)
plt.xlabel("date_block_num")
plt.ylabel("item_cnt_month")
plt.title("cat2")

In [ ]:
a=matrix.groupby(["date_block_num","item_category_id"])["item_cnt_month"].sum().reset_index()
sns.lineplot(x="date_block_num",y="item_cnt_month",data=a,hue=a.item_category_id)
plt.xlabel("date_block_num")
plt.ylabel("item_cnt_month")
plt.title("item_category_id")

In [ ]:
matrix["month"]=matrix.date_block_num%12
matrix["year"]=(matrix.date_block_num/12).apply(lambda x: int(x))

In [ ]:
## functoin for p,t ,F values
from scipy.special import ndtr
from sklearn.feature_selection import SelectKBest, f_classif
def tpf_of(matrix):
    selector = SelectKBest(f_classif, k=5)
    X_train=matrix.query("date_block_num<33").drop("item_cnt_month",axis=1)
    y_train=matrix.query("date_block_num<33")["item_cnt_month"]
    selector.fit(X_train, y_train)
    F=selector.scores_
    t=np.sqrt(selector.scores_)
    p=1-ndtr(t)
    return(pd.DataFrame(data=np.vstack((F,t,p)).transpose(),columns=["F","t","p"],index=X_train.columns))

# More Features

In [ ]:
#shop count
a=matrix.query("date_block_num<34").groupby("shop_id").agg({"item_cnt_month":["mean"]})
a.columns=["shop_id_avg"]
matrix['shop_id_avg']=matrix["shop_id"].map(a['shop_id_avg'])

In [ ]:
# item count MOST USEFUL
a=matrix.query("date_block_num<34").groupby("item_id").agg({"item_cnt_month":["mean"]})
a.columns=["item_id_avg"]
matrix['item_id_avg']=matrix["item_id"].map(a['item_id_avg'])

# item category
a=matrix.query("date_block_num<34").groupby("item_category_id").agg({"item_cnt_month":["mean"]})
a.columns=["item_category_id_avg"]
matrix['item_category_id_avg']=matrix["item_category_id"].map(a['item_category_id_avg'])

In [ ]:
## date item count
a=matrix.query("date_block_num<34").groupby(["date_block_num","item_id"]).agg({"item_cnt_month":["mean"]})
a.columns=["date_block_item_id_num_avg"]

matrix["date_block_item_id_num_avg"]=pd.merge(matrix,a.reset_index(),
                                              on=["date_block_num","item_id"],
                                              how="left")["date_block_item_id_num_avg"]

a.reset_index(inplace=True)
a["date_block_num"]=a["date_block_num"]+1
matrix=pd.merge(matrix,a.rename(columns={"date_block_item_id_num_avg":"date_block_item_id_num_avg_lag_"+str(1)},inplace=False)
         ,on=["date_block_num","item_id"],how="left")

a["date_block_num"]=a["date_block_num"]+1
matrix=pd.merge(matrix,a.rename(columns={"date_block_item_id_num_avg":"date_block_item_id_num_avg_lag_"+str(2)},inplace=False)
         ,on=["date_block_num","item_id"],how="left")

a["date_block_num"]=a["date_block_num"]+1
matrix=pd.merge(matrix,a.rename(columns={"date_block_item_id_num_avg":"date_block_item_id_num_avg_lag_"+str(3)},inplace=False)
         ,on=["date_block_num","item_id"],how="left")

a["date_block_num"]=a["date_block_num"]+3
matrix=pd.merge(matrix,a.rename(columns={"date_block_item_id_num_avg":"date_block_item_id_num_avg_lag_"+str(6)},inplace=False)
         ,on=["date_block_num","item_id"],how="left")

a["date_block_num"]=a["date_block_num"]+6
matrix=pd.merge(matrix,a.rename(columns={"date_block_item_id_num_avg":"date_block_item_id_num_avg_lag_"+str(12)},inplace=False)
         ,on=["date_block_num","item_id"],how="left")

In [ ]:
## date shop count
a=matrix.query("date_block_num<34").groupby(["date_block_num","shop_id"]).agg({"item_cnt_month":["mean"]})
a.columns=["date_block_shop_id_num_avg"]

matrix["date_block_shop_id_num_avg"]=pd.merge(matrix,a.reset_index(),
                                              on=["date_block_num","shop_id"],
                                              how="left")["date_block_shop_id_num_avg"]

a.reset_index(inplace=True)
a["date_block_num"]=a["date_block_num"]+1
matrix=pd.merge(matrix,a.rename(columns={"date_block_shop_id_num_avg":"date_block_shop_id_num_avg_lag_"+str(1)},inplace=False)
         ,on=["date_block_num","shop_id"],how="left")

a["date_block_num"]=a["date_block_num"]+1
matrix=pd.merge(matrix,a.rename(columns={"date_block_shop_id_num_avg":"date_block_shop_id_num_avg_lag_"+str(2)},inplace=False)
         ,on=["date_block_num","shop_id"],how="left")

a["date_block_num"]=a["date_block_num"]+1
matrix=pd.merge(matrix,a.rename(columns={"date_block_shop_id_num_avg":"date_block_shop_id_num_avg_lag_"+str(3)},inplace=False)
         ,on=["date_block_num","shop_id"],how="left")

a["date_block_num"]=a["date_block_num"]+3
matrix=pd.merge(matrix,a.rename(columns={"date_block_shop_id_num_avg":"date_block_shop_id_num_avg_lag_"+str(6)},inplace=False)
         ,on=["date_block_num","shop_id"],how="left")

a["date_block_num"]=a["date_block_num"]+6
matrix=pd.merge(matrix,a.rename(columns={"date_block_shop_id_num_avg":"date_block_shop_id_num_avg_lag_"+str(12)},inplace=False)
         ,on=["date_block_num","shop_id"],how="left")

In [ ]:
## item price
a=sales_train.query("date_block_num<34").groupby("item_id").agg({"item_price":["mean"]})
a.columns=["mean_item_price"]
matrix["item_price"]=matrix["item_id"].map(a["mean_item_price"])

In [ ]:
## date shop revenue
a=sales_train.query("date_block_num<34").groupby(["date_block_num","shop_id"]).agg({"revenue":["sum"]})
a.columns=["date_block_shop_id_revenue_avg"]
a.reset_index(inplace=True)

matrix["date_block_shop_id_revenue_avg"]=pd.merge(matrix,a,
                                              on=["date_block_num","shop_id"],
                                              how="left")["date_block_shop_id_revenue_avg"]

a["date_block_num"]=a["date_block_num"]+1

matrix["date_block_shop_id_revenue_avg_lag_"+str(1)]=pd.merge(matrix,a.rename(columns={"date_block_shop_id_revenue_avg":"date_block_shop_id_revenue_avg_lag_"+str(1)}),
                                              on=["date_block_num","shop_id"],
                                              how="left")["date_block_shop_id_revenue_avg_lag_"+str(1)]

a["date_block_num"]=a["date_block_num"]+1

matrix["date_block_shop_id_revenue_avg_lag_"+str(2)]=pd.merge(matrix,a.rename(columns={"date_block_shop_id_revenue_avg":"date_block_shop_id_revenue_avg_lag_"+str(2)}),
                                              on=["date_block_num","shop_id"],
                                              how="left")["date_block_shop_id_revenue_avg_lag_"+str(2)]

a["date_block_num"]=a["date_block_num"]+1

matrix["date_block_shop_id_revenue_avg_lag_"+str(3)]=pd.merge(matrix,a.rename(columns={"date_block_shop_id_revenue_avg":"date_block_shop_id_revenue_avg_lag_"+str(3)}),
                                              on=["date_block_num","shop_id"],
                                              how="left")["date_block_shop_id_revenue_avg_lag_"+str(3)]

In [ ]:
## date item count
a=matrix.query("date_block_num<34").groupby(["date_block_num","item_category_id"]).agg({"item_cnt_month":["mean"]})
a.columns=["date_block_item_category_id_num_avg"]

matrix["date_block_item_category_id_num_avg"]=pd.merge(matrix,a.reset_index(),
                                              on=["date_block_num","item_category_id"],
                                              how="left")["date_block_item_category_id_num_avg"]

a.reset_index(inplace=True)
a["date_block_num"]=a["date_block_num"]+1
matrix=pd.merge(matrix,a.rename(columns={"date_block_item_category_id_num_avg":"date_block_item_category_id_num_avg_lag_"+str(1)},inplace=False)
         ,on=["date_block_num","item_category_id"],how="left")

a["date_block_num"]=a["date_block_num"]+1
matrix=pd.merge(matrix,a.rename(columns={"date_block_item_category_id_num_avg":"date_block_item_category_id_num_avg_lag_"+str(2)},inplace=False)
         ,on=["date_block_num","item_category_id"],how="left")

a["date_block_num"]=a["date_block_num"]+1
matrix=pd.merge(matrix,a.rename(columns={"date_block_item_category_id_num_avg":"date_block_item_category_id_num_avg_lag_"+str(3)},inplace=False)
         ,on=["date_block_num","item_category_id"],how="left")

a["date_block_num"]=a["date_block_num"]+3
matrix=pd.merge(matrix,a.rename(columns={"date_block_item_category_id_num_avg":"date_block_item_category_id_num_avg_lag_"+str(6)},inplace=False)
         ,on=["date_block_num","item_category_id"],how="left")

a["date_block_num"]=a["date_block_num"]+6
matrix=pd.merge(matrix,a.rename(columns={"date_block_item_category_id_num_avg":"date_block_item_category_id_num_avg_lag_"+str(12)},inplace=False)
         ,on=["date_block_num","item_category_id"],how="left")

In [ ]:
del sales_train, shops,items, item_categories,a
gc.collect()

In [ ]:
matrix["date_block_num"]=matrix["date_block_num"].astype("int8")
matrix["shop_id"]=matrix["shop_id"].astype("int8")
matrix["item_id"]=matrix["item_id"].astype("int16")
matrix["item_cnt_month"]=matrix["item_cnt_month"].astype("int8")
matrix["item_category_id"]=matrix["item_category_id"].astype("int8")
matrix["shop_city"]=matrix["shop_city"].astype("int8")
matrix["shop_type"]=matrix["shop_type"].astype("int8")
matrix["cat1"]=matrix["cat1"].astype("int8")
matrix["cat2"]=matrix["cat2"].astype("int8")
matrix["month"]=matrix["month"].astype("int8")
matrix["year"]=matrix["year"].astype("int8")
matrix["item_price"]=matrix["item_price"].astype("float16")
matrix["date_block_shop_id_revenue_avg"]=matrix["date_block_shop_id_revenue_avg"].astype("float32")
matrix["date_block_shop_id_revenue_avg_lag_1"]=matrix["date_block_shop_id_revenue_avg_lag_1"].astype("float32")
matrix["date_block_shop_id_revenue_avg_lag_2"]=matrix["date_block_shop_id_revenue_avg_lag_2"].astype("float32")
matrix["date_block_shop_id_revenue_avg_lag_3"]=matrix["date_block_shop_id_revenue_avg_lag_3"].astype("float32")

CAT1 and CAT2

In [ ]:
a=matrix.query("date_block_num<34").groupby("cat1").agg({"item_cnt_month":["mean"]})
a.columns=["cat1_avg"]
matrix['cat1_avg']=matrix["cat1"].map(a['cat1_avg'])
matrix['cat1_avg']=matrix['cat1_avg'].astype("float16")
print("done")

a=matrix.query("date_block_num<34").groupby("cat2").agg({"item_cnt_month":["mean"]})
a.columns=["cat2_avg"]
matrix['cat2_avg']=matrix["cat2"].map(a['cat2_avg'])
matrix['cat2_avg']=matrix['cat2_avg'].astype("float16")
print("done")


a=matrix.query("date_block_num<34").groupby(["date_block_num","cat1"]).agg({"item_cnt_month":["mean"]})
a.columns=["date_block_cat1_num_avg"]

matrix["date_block_cat1_num_avg"]=pd.merge(matrix,a.reset_index(),
                                              on=["date_block_num","cat1"],
                                              how="left")["date_block_cat1_num_avg"]
matrix['date_block_cat1_num_avg']=matrix['date_block_cat1_num_avg'].astype("float16")


a.reset_index(inplace=True)
a["date_block_num"]=a["date_block_num"]+1
matrix=pd.merge(matrix,a.rename(columns={"date_block_cat1_num_avg":"date_block_cat1_num_avg_lag_"+str(1)},inplace=False)
         ,on=["date_block_num","cat1"],how="left")
matrix['date_block_cat1_num_avg_lag_1']=matrix['date_block_cat1_num_avg_lag_1'].astype("float16")

a["date_block_num"]=a["date_block_num"]+1
matrix=pd.merge(matrix,a.rename(columns={"date_block_cat1_num_avg":"date_block_cat1_num_avg_lag_"+str(2)},inplace=False)
         ,on=["date_block_num","cat1"],how="left")
matrix['date_block_cat1_num_avg_lag_2']=matrix['date_block_cat1_num_avg_lag_2'].astype("float16")

a["date_block_num"]=a["date_block_num"]+1
matrix=pd.merge(matrix,a.rename(columns={"date_block_cat1_num_avg":"date_block_cat1_num_avg_lag_"+str(3)},inplace=False)
         ,on=["date_block_num","cat1"],how="left")
matrix['date_block_cat1_num_avg_lag_3']=matrix['date_block_cat1_num_avg_lag_3'].astype("float16")

print("done")



a=matrix.query("date_block_num<34").groupby(["date_block_num","cat2"]).agg({"item_cnt_month":["mean"]})
a.columns=["date_block_cat2_num_avg"]

matrix["date_block_cat2_num_avg"]=pd.merge(matrix,a.reset_index(),
                                              on=["date_block_num","cat2"],
                                              how="left")["date_block_cat2_num_avg"]
matrix['date_block_cat2_num_avg']=matrix['date_block_cat2_num_avg'].astype("float16")

a.reset_index(inplace=True)
a["date_block_num"]=a["date_block_num"]+1
matrix=pd.merge(matrix,a.rename(columns={"date_block_cat2_num_avg":"date_block_cat2_num_avg_lag_"+str(1)},inplace=False)
         ,on=["date_block_num","cat2"],how="left")
matrix['date_block_cat2_num_avg_lag_1']=matrix['date_block_cat2_num_avg_lag_1'].astype("float16")

a["date_block_num"]=a["date_block_num"]+1
matrix=pd.merge(matrix,a.rename(columns={"date_block_cat2_num_avg":"date_block_cat2_num_avg_lag_"+str(2)},inplace=False)
         ,on=["date_block_num","cat2"],how="left")
matrix['date_block_cat2_num_avg_lag_1']=matrix['date_block_cat2_num_avg_lag_2'].astype("float16")

a["date_block_num"]=a["date_block_num"]+1
matrix=pd.merge(matrix,a.rename(columns={"date_block_cat2_num_avg":"date_block_cat2_num_avg_lag_"+str(3)},inplace=False)
         ,on=["date_block_num","cat2"],how="left")
matrix['date_block_cat2_num_avg_lag_1']=matrix['date_block_cat2_num_avg_lag_3'].astype("float16")

print("done")

In [ ]:
a=matrix.query("date_block_num<34").groupby("shop_type").agg({"item_cnt_month":["mean"]})
a.columns=["shop_type_avg"]
matrix['shop_type_avg']=matrix["shop_type"].map(a['shop_type_avg'])
matrix['shop_type_avg']=matrix['shop_type_avg'].astype("float16")
print("done")

a=matrix.query("date_block_num<34").groupby("shop_city").agg({"item_cnt_month":["mean"]})
a.columns=["shop_city_avg"]
matrix['shop_city_avg']=matrix["shop_city"].map(a['shop_city_avg'])
matrix['shop_city_avg']=matrix['shop_city_avg'].astype("float16")
print("done")


a=matrix.query("date_block_num<34").groupby(["date_block_num","shop_type"]).agg({"item_cnt_month":["mean"]})
a.columns=["date_block_shop_type_num_avg"]

matrix["date_block_shop_type_num_avg"]=pd.merge(matrix,a.reset_index(),
                                              on=["date_block_num","shop_type"],
                                              how="left")["date_block_shop_type_num_avg"]
matrix['date_block_shop_type_num_avg']=matrix['date_block_shop_type_num_avg'].astype("float16")
print("done")



a=matrix.query("date_block_num<34").groupby(["date_block_num","shop_city"]).agg({"item_cnt_month":["mean"]})
a.columns=["date_block_shop_city_num_avg"]

matrix["date_block_shop_city_num_avg"]=pd.merge(matrix,a.reset_index(),
                                              on=["date_block_num","shop_city"],
                                              how="left")["date_block_shop_city_num_avg"]
matrix['date_block_shop_city_num_avg']=matrix['date_block_shop_city_num_avg'].astype("float16")
print("done")

In [ ]:
a=matrix.query("date_block_num<34").groupby(["item_id","shop_id"]).agg({"item_cnt_month":["mean"]})
a.columns=["item_shop_id_num_avg"]

matrix["item_shop_id_num_avg"]=pd.merge(matrix,a.reset_index(),
                                              on=["item_id","shop_id"],
                                              how="left")["item_shop_id_num_avg"]

a=matrix.query("date_block_num<34").groupby(["item_category_id","shop_id"]).agg({"item_cnt_month":["mean"]})
a.columns=["item_category_shop_id_num_avg"]

matrix["item_category_shop_id_num_avg"]=pd.merge(matrix,a.reset_index(),
                                              on=["item_category_id","shop_id"],
                                              how="left")["item_category_shop_id_num_avg"]


In [ ]:
a=matrix.groupby(["date_block_num","shop_id","item_id"])["item_price"].mean().reset_index()
a_lag1=a.copy()
a_lag1["date_block_num"]=a["date_block_num"]+1
a=pd.merge(a,a_lag1,on=["date_block_num","shop_id","item_id"],how="left",suffixes=('','_lag1'))

a_lag2=a_lag1.copy()
a_lag2["date_block_num"]=a_lag1["date_block_num"]+1
a=pd.merge(a,a_lag2,on=["date_block_num","shop_id","item_id"],how="left",suffixes=('','_lag2'))

a_lag3=a_lag2.copy()
a_lag3["date_block_num"]=a_lag2["date_block_num"]+1
a=pd.merge(a,a_lag3,on=["date_block_num","shop_id","item_id"],how="left",suffixes=('','_lag3'))

a_lag6=a_lag3.copy()
a_lag6["date_block_num"]=a_lag3["date_block_num"]+3
a=pd.merge(a,a_lag6,on=["date_block_num","shop_id","item_id"],how="left",suffixes=('','_lag6'))

a_lag12=a_lag6.copy()
a_lag12["date_block_num"]=a_lag6["date_block_num"]+6
a=pd.merge(a,a_lag12,on=["date_block_num","shop_id","item_id"],how="left",suffixes=('','_lag12'))


a_mean=matrix.groupby(["shop_id","item_id"])["item_price"].mean().reset_index()
a=pd.merge(a,a_mean,on=["shop_id","item_id"],how="left",suffixes=('','_mean'))

a["price_delta1"]=(a.item_price-a.item_price_lag1)/a.item_price_mean
a["price_delta2"]=(a.item_price-a.item_price_lag2)/a.item_price_mean
a["price_delta3"]=(a.item_price-a.item_price_lag3)/a.item_price_mean
a["price_delta6"]=(a.item_price-a.item_price_lag6)/a.item_price_mean
a["price_delta12"]=(a.item_price-a.item_price_lag12)/a.item_price_mean

a.drop(["item_price","item_price_lag1","item_price_lag2","item_price_lag3","item_price_lag6","item_price_lag12","item_price_mean"],axis=1,inplace=True)
a.date_block_num=a.date_block_num.astype("int8")
a.shop_id=a.shop_id.astype("int8")
a.item_id=a.item_id.astype("int16")
a.price_delta1=a.price_delta1.astype("float16")
a.price_delta2=a.price_delta2.astype("float16")
a.price_delta3=a.price_delta3.astype("float16")
a.price_delta6=a.price_delta6.astype("float16")
a.price_delta12=a.price_delta12.astype("float16")

price_trend=a

del a,a_lag1,a_lag2,a_lag3,a_lag6,a_lag12,a_mean
gc.collect()

In [ ]:
matrix["item_max_price"]=matrix["item_id"].map(max_price)

matrix["discount"]=(matrix["item_max_price"]-matrix["item_price"])*100/matrix["item_max_price"]
matrix["discount"]=matrix["discount"].apply("int8")

## for price delta
matrix=pd.merge(matrix,price_trend,on=["date_block_num","shop_id","item_id"],how="left")

In [ ]:
pd.set_option("max_columns",500)

In [ ]:
matrix.query('date_block_num==34')

In [ ]:
ts=time.time()
cache = {}
matrix['item_shop_last_sale'] = -1
matrix['item_shop_last_sale'] = matrix['item_shop_last_sale'].astype(np.int8)
for idx, row in matrix.iterrows():    
    key = str(row.item_id)+' '+str(row.shop_id)
    if key not in cache:
        if row.item_cnt_month!=0:
            cache[key] = row.date_block_num
    else:
        last_date_block_num = cache[key]
        matrix.at[idx, 'item_shop_last_sale'] = row.date_block_num - last_date_block_num
        cache[key] = row.date_block_num
        
print("time taken", time.time()-ts)
del cache
gc.collect()

In [ ]:
ts=time.time()
matrix["item_last_sale"]=-1
matrix["item_last_sale"]=matrix["item_last_sale"].apply("int8")

cache={}

for idx,row in matrix.iterrows():
    key=str(row.item_id)
    if key not in cache.keys():
        if row.item_cnt_month!=0:
            cache[key]=row.date_block_num
    else:
        last_date_block_num=cache[key]
        if row.date_block_num>last_date_block_num:
            matrix.at[idx,'item_last_sale']=row.date_block_num-last_date_block_num
            cache[key]=row.date_block_num

matrix["item_last_sale"]=matrix["item_last_sale"].apply("int8")
print("time taken=",time.time()-ts)

In [ ]:
matrix["item_shop_first_sale"]=matrix['date_block_num']-matrix.groupby(['item_id','shop_id'])['date_block_num'].transform('min')
matrix["item_shop_first_sale"]=matrix["item_shop_first_sale"].apply('int8')

matrix['item_first_sale']=matrix['date_block_num']-matrix.groupby('item_id')['date_block_num'].transform('min')
matrix['item_first_sale']=matrix['item_first_sale'].apply('int8')

In [ ]:
matrix.drop(["date_block_item_id_num_avg","date_block_shop_id_num_avg","date_block_shop_id_revenue_avg",
            "date_block_item_category_id_num_avg","date_block_cat1_num_avg","date_block_cat2_num_avg",
             "date_block_shop_type_num_avg","date_block_shop_city_num_avg"],axis=1,inplace=True)

In [ ]:
matrix.to_pickle("matrix.pkl")

In [ ]:
matrix=pd.read_pickle("../input/sales-train/matrix.pkl")

In [ ]:
%%time
# date count
a=matrix.query("date_block_num<34").groupby("date_block_num").agg({"item_cnt_month":["mean"]})
a.columns=["date_block_num_avg"]

matrix["date_block_num_avg"]=pd.merge(matrix,a.reset_index(),
                                              on="date_block_num",
                                              how="left")["date_block_num_avg"]
matrix["date_block_num_avg"]=matrix["date_block_num_avg"].astype("float16")

a.reset_index(inplace=True)
a["date_block_num"]=a["date_block_num"]+1
matrix=pd.merge(matrix,a.rename(columns={"date_block_num_avg":"date_block_num_avg_lag_"+str(1)},inplace=False)
         ,on="date_block_num",how="left")
matrix["date_block_num_avg_lag_1"]=matrix["date_block_num_avg_lag_1"].astype("float16")


a["date_block_num"]=a["date_block_num"]+1
matrix=pd.merge(matrix,a.rename(columns={"date_block_num_avg":"date_block_num_avg_lag_"+str(2)},inplace=False)
         ,on="date_block_num",how="left")
matrix["date_block_num_avg_lag_2"]=matrix["date_block_num_avg_lag_2"].astype("float16")


a["date_block_num"]=a["date_block_num"]+1
matrix=pd.merge(matrix,a.rename(columns={"date_block_num_avg":"date_block_num_avg_lag_"+str(3)},inplace=False)
         ,on="date_block_num",how="left")
matrix["date_block_num_avg_lag_3"]=matrix["date_block_num_avg_lag_3"].astype("float16")

In [ ]:
%%time
## date item shop count
a=matrix.query("date_block_num<34").groupby(["date_block_num","item_id","shop_id"]).agg({"item_cnt_month":["mean"]})
a.columns=["date_block_item_shop_num_avg"]

matrix["date_block_item_shop_num_avg"]=pd.merge(matrix,a.reset_index(),
                                              on=["date_block_num","item_id","shop_id"],
                                              how="left")["date_block_item_shop_num_avg"]
matrix["date_block_item_shop_num_avg"]=matrix["date_block_item_shop_num_avg"].astype('float16')

a.reset_index(inplace=True)
a["date_block_num"]=a["date_block_num"]+1
matrix=pd.merge(matrix,a.rename(columns={"date_block_item_shop_num_avg":"date_block_item_shop_num_avg_lag_"+str(1)},inplace=False)
         ,on=["date_block_num","item_id","shop_id"],how="left")
matrix["date_block_item_shop_num_avg_lag_"+str(1)]=matrix["date_block_item_shop_num_avg_lag_"+str(1)].astype('float16')


a["date_block_num"]=a["date_block_num"]+1
matrix=pd.merge(matrix,a.rename(columns={"date_block_item_shop_num_avg":"date_block_item_shop_num_avg_lag_"+str(2)},inplace=False)
         ,on=["date_block_num","item_id","shop_id"],how="left")
matrix["date_block_item_shop_num_avg_lag_"+str(2)]=matrix["date_block_item_shop_num_avg_lag_"+str(2)].astype('float16')


a["date_block_num"]=a["date_block_num"]+1
matrix=pd.merge(matrix,a.rename(columns={"date_block_item_shop_num_avg":"date_block_item_shop_num_avg_lag_"+str(3)},inplace=False)
         ,on=["date_block_num","item_id","shop_id"],how="left")
matrix["date_block_item_shop_num_avg_lag_"+str(3)]=matrix["date_block_item_shop_num_avg_lag_"+str(3)].astype('float16')


matrix.drop("date_block_item_shop_num_avg",axis=1,inplace=True)

In [ ]:
%%time
## date item category shop count
a=matrix.query("date_block_num<34").groupby(["date_block_num","item_category_id","shop_id"]).agg({"item_cnt_month":["mean"]})
a.columns=["date_block_item_category_shop_num_avg"]

matrix["date_block_item_category_shop_num_avg"]=pd.merge(matrix,a.reset_index(),
                                              on=["date_block_num","item_category_id","shop_id"],
                                              how="left")["date_block_item_category_shop_num_avg"]
matrix["date_block_item_category_shop_num_avg"]=matrix["date_block_item_category_shop_num_avg"].astype('float16')


a.reset_index(inplace=True)
a["date_block_num"]=a["date_block_num"]+1
matrix=pd.merge(matrix,a.rename(columns={"date_block_item_category_shop_num_avg":"date_block_item_category_shop_num_avg_lag_"+str(1)},inplace=False)
         ,on=["date_block_num","item_category_id","shop_id"],how="left")
matrix["date_block_item_category_shop_num_avg_lag_"+str(1)]=matrix["date_block_item_category_shop_num_avg_lag_"+str(1)].astype('float16')


a["date_block_num"]=a["date_block_num"]+1
matrix=pd.merge(matrix,a.rename(columns={"date_block_item_category_shop_num_avg":"date_block_item_category_shop_num_avg_lag_"+str(2)},inplace=False)
         ,on=["date_block_num","item_category_id","shop_id"],how="left")
matrix["date_block_item_category_shop_num_avg_lag_"+str(2)]=matrix["date_block_item_category_shop_num_avg_lag_"+str(2)].astype('float16')


a["date_block_num"]=a["date_block_num"]+1
matrix=pd.merge(matrix,a.rename(columns={"date_block_item_category_shop_num_avg":"date_block_item_category_shop_num_avg_lag_"+str(3)},inplace=False)
         ,on=["date_block_num","item_category_id","shop_id"],how="left")
matrix["date_block_item_category_shop_num_avg_lag_"+str(3)]=matrix["date_block_item_category_shop_num_avg_lag_"+str(3)].astype('float16')


matrix.drop("date_block_item_category_shop_num_avg",axis=1,inplace=True)

matrix.drop(cols_to_drop,axis=1,inplace=True)

In [ ]:
matrix=matrix.query("date_block_num>11")
matrix.fillna(0,inplace=True)

In [ ]:
matrix.info()

In [ ]:
X_train=matrix.drop("item_cnt_month",axis=1).query("date_block_num<33")
y_train=matrix.query("date_block_num<33")["item_cnt_month"]
X_val=matrix.drop("item_cnt_month",axis=1).query("date_block_num==33")
y_val=matrix.query("date_block_num==33")["item_cnt_month"]
X_test=matrix.drop("item_cnt_month",axis=1).query("date_block_num==34")

del matrix
gc.collect()

# MODELING

# RandomForest

ENSEMBLING

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
## ENSEMBLEE
ts=time.time()
clf=RandomForestRegressor(max_features="sqrt",n_estimators=500,criterion="mse",max_samples=0.6,
                               random_state=0,max_depth=11,n_jobs=-1,verbose=True)
clf.fit(X_train,y_train)

print("time taken= ", time.time()-ts)

In [ ]:
y_pred_RF=clf.predict(X_val)
test_preds_rf=clf.predict(X_test.fillna(0))
mse=mean_squared_error(y_pred_RF,y_val)
print("MSE=",mse)

In [ ]:
test_preds=clf.predict(X_test.fillna(0))
submission=pd.DataFrame({"ID":test.index,"item_cnt_month":test_preds})
submission.to_csv("submission_RF.csv",index=False)

In [ ]:
y_pred_RF=pd.DataFrame({"y_pred_RF":y_pred_RF})
test_preds_rf=pd.DataFrame({"test_preds_rf":test_preds_rf})

y_pred_RF.to_csv("y_pred_RF.csv")
test_preds_rf.to_csv("test_preds_rf.csv")

# XGB

In [ ]:
ts = time.time()

model = XGBRegressor(
    max_depth=9,
    n_estimators=5000,
    colsample_bytree=0.12403473458920847, 
    subsample=0.6,
    min_child_weight=300,
    eta=0.01,
    seed=0)

model.fit(
    X_train, 
    y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, y_train), (X_val, y_val)], 
    verbose=True, 
    early_stopping_rounds = 10)

time.time() - ts

eta=0.1, trees=654

In [ ]:
test_preds=model.predict(X_test.fillna(0))
submission=pd.DataFrame({"ID":test.index,"item_cnt_month":test_preds})
submission.to_csv("submission.csv",index=False)

In [ ]:
from xgboost import plot_importance

def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

plot_features(model,(10,14))

In [ ]:
y_pred_XGB=model.predict(X_val)
test_preds_xgb = model.predict(X_test.fillna(0))

In [ ]:
test_preds = model.predict(X_test.fillna(0))
test_preds[test_preds<0]=0
test_preds[test_preds>20]=20
submission=pd.DataFrame({"ID":test.index,"item_cnt_month":test_preds})
submission.to_csv("submission_XGBM.csv",index=False)
submission_XGBM=submission

# LIGHT GBM

In [ ]:
import lightgbm as lgb

In [ ]:
ts=time.time()
dtrain = lgb.Dataset(X_train, label=y_train)
dvalid = lgb.Dataset(X_val, label=y_val)

param = {'objective': 'regression',"eta":0.01,"random_state":0,"max_depth":2,"subsample":0.67,
         "feature_fraction_bynode":0.12403473458920847,"min_child_weight":300}
param['metric'] = 'mse'
num_round = 10000
bst = lgb.train(param, dtrain, num_round, valid_sets=[dvalid], early_stopping_rounds=10, 
                verbose_eval=True)
print("time taken:",time.time()-ts)

In [ ]:
y_pred_LGB=bst.predict(X_val)
test_preds_lgb = bst.predict(X_test.fillna(0))

In [ ]:
test_preds = bst.predict(X_test.fillna(0))
test_preds[test_preds<0]=0
submission=pd.DataFrame({"ID":test.index,"item_cnt_month":test_preds.round()})
submission.to_csv("submission_LGBM.csv",index=False)
submission_LGBM=submission

In [ ]:
1/np.sqrt(X_train.shape[1])

In [ ]:
y_train.mean()

In [ ]:
test_preds.mean()

In [ ]:
from lightgbm import plot_importance

def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

plot_features(bst,(10,14))

In [ ]:
cols_to_drop=["item_max_price","item_price","shop_type_avg",'shop_type',"item_category_id",
             "discount",'date_block_shop_id_revenue_avg_lag_2',"date_block_shop_id_revenue_avg_lag_1",
              'date_block_shop_id_revenue_avg_lag_3',"cat1","cat2","cat1_avg",
              "date_block_cat1_num_avg_lag_1","date_block_cat1_num_avg_lag_2","date_block_cat1_num_avg_lag_3",
              "date_block_cat2_num_avg_lag_2","date_block_cat2_num_avg_lag_1",'date_block_cat2_num_avg_lag_3',
              "item_id",'shop_city','shop_city_avg','shop_id']

In [ ]:
plt.scatter(np.arange(len(test_preds)),test_preds)

In [ ]:
sns.boxplot(y=test_preds)

# ENSEMBLING

In [ ]:
plt.scatter(test_preds_rf,test_preds_xgb)

In [ ]:
submission["item_cnt_month"].max()

In [ ]:
y_pred_RF["y_pred_RF"]

In [ ]:
y_pred_df=pd.DataFrame({"y_pred_RF":y_pred_RF,"y_pred_XGB":y_pred_XGB,"y_val":y_val})
y_test_df=pd.DataFrame({"test_preds_rf":test_preds_rf,"test_preds_xgb":test_preds_xgb})

In [ ]:
y_pred_df.to_pickle("y_preds.pkl")
y_test_df.to_pickle("y_test.pkl")

In [ ]:
print("MSE for mean ",mean_squared_error(y_pred_df[["y_pred_RF","y_pred_XGB"]].mean(axis=1),y_pred_df["y_val"]))

In [ ]:
lm=LinearRegression()
lm.fit(y_pred_df[["y_pred_RF","y_pred_XGB"]],y_pred_df["y_val"])
test_preds_lvl2=lm.predict(y_test_df)
test_preds_lvl2[test_preds_lvl2<0]=0
submission=pd.DataFrame({"ID":test.index,"item_cnt_month":test_preds_lvl2})
submission.to_csv("submission.csv",index=False)

In [ ]:
r2_score(y_pred_df["y_val"],lm.predict(y_pred_df[["y_pred_RF","y_pred_XGB"]]))

In [ ]:
r2_score(y_pred_df["y_val"],(y_pred_df["y_pred_RF"]+y_pred_df["y_pred_XGB"])/2)

In [ ]:
test_preds_lgb[test_preds_lgb<0]=0
test_preds_xgb[test_preds_xgb<0]=0
test_preds_rf[test_preds_rf<0]=0

submission=pd.DataFrame({"ID":test.index,"item_cnt_month":(test_preds_xgb+test_preds_lgb)/3})
submission.to_csv("submission.csv",index=False)

In [ ]:
sns.scatterplot(np.arange(len(test_preds_xgb)),test_preds_xgb)
sns.scatterplot(np.arange(len(test_preds_lgb)),test_preds_lgb)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
print("R2 score for DT",r2_score(y_val,y_pred_DT))
print("R2 score for DT",r2_score(y_val,y_pred_RF))
print("R2 score for DT",r2_score(y_val,y_pred_LGB))

In [ ]:
selector = SelectKBest(f_classif,k=3)
selector.fit(y_pred_df[["y_pred_DT","y_pred_RF","y_pred_LGB"]],y_pred_df["y_val"])
F=selector.scores_
t=np.sqrt(selector.scores_)
p=1-ndtr(t)
pd.DataFrame(data=np.vstack((F,t,p)).transpose(),columns=["F","t","p"],index=y_pred_df[["y_pred_DT","y_pred_RF","y_pred_LGB"]].columns)

In [ ]:
y_pred_df[["y_pred_DT","y_pred_RF","y_pred_LGB"]]

In [ ]:
y_pred_df